In [1]:
# Instal Pustaka
!pip install streamlit pandas joblib matplotlib seaborn scikit-learn pyngrok

# Instal Node.js (untuk stabilitas Colab)
!npm install -g localtunnel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 117.3 MB/s eta 0:00:00
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧
added 22 packages in 3s
⠧
⠧3 packages are looking for funding
⠧  run `npm fund` for details
⠧

Pelatihan dan Penyimpanan Model Random Forest

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import joblib
import os

# --- A. Konfigurasi File ---
DATA_FILE = 'dataset_banjir.csv'
MODEL_FILE = 'model_prediksi_banjir.pkl'

def load_and_clean_data(file_path):
    df = pd.DataFrame()

    try:
        df = pd.read_csv(file_path, sep=';')
    except Exception:
        return pd.DataFrame()

    # 1. Bersihkan Kolom Kosong di Akhir
    df = df.dropna(axis=1, how='all')

    # 2. Hapus Baris Kosong di Awal
    df = df.dropna(axis=0, how='all').reset_index(drop=True)

    if df.empty:
        return pd.DataFrame()

    df.columns = df.iloc[0]
    df = df[1:].reset_index(drop=True)

    # -----------------------------------------------------------------
    # PENTING: Seleksi Kolom - Memilih HANYA 4 kolom pertama dari 10 yang tersisa
    # Ini menjamin jumlah kolom adalah 4, sesuai kebutuhan model
    df = df.iloc[:, :4]
    # -----------------------------------------------------------------

    try:
        df.columns = ['Kabupaten_Kota', 'Banjir', 'Banjir_Bandang', 'Curah_Hujan']
    except ValueError:
        print(f"DEBUG: Masih ada masalah setelah seleksi. Kolom yang tersisa: {len(df.columns)}")
        return pd.DataFrame()

    # 4. Konversi Tipe Data dan Koma Desimal
    kolom_numerik = ['Banjir', 'Banjir_Bandang', 'Curah_Hujan']
    for col in kolom_numerik:
        if df[col].dtype == 'object':
             df[col] = df[col].astype(str).str.replace(',', '.', regex=False)
        df[col] = pd.to_numeric(df[col], errors='coerce')

    df[kolom_numerik] = df[kolom_numerik].fillna(0)
    df['Total_Bencana'] = df['Banjir'] + df['Banjir_Bandang']

    return df

df_model = load_and_clean_data(DATA_FILE)

if df_model.empty:
    print(f"❌ GAGAL KRITIS: Tidak dapat memuat atau membersihkan data dari {DATA_FILE}")
else:
    print("✅ Data berhasil dimuat dan dibersihkan.")
    # --- C. Pelatihan Model Random Forest Regressor ---
    X = df_model[['Curah_Hujan']]
    Y = df_model['Total_Bencana']

    rfr_model = RandomForestRegressor(n_estimators=100, random_state=42)
    rfr_model.fit(X, Y)

    # --- D. Simpan Model ---
    joblib.dump(rfr_model, MODEL_FILE)
    print(f"✅ Model Random Forest Regressor disimpan sebagai '{MODEL_FILE}'")

✅ Data berhasil dimuat dan dibersihkan.
✅ Model Random Forest Regressor disimpan sebagai 'model_prediksi_banjir.pkl'


In [10]:
%%writefile app.py

import streamlit as st
import pandas as pd
import joblib
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# --- 1. Konfigurasi Awal ---
st.set_page_config(
    page_title="Dashboard Prediksi Bencana Banjir",
    layout="wide",
    initial_sidebar_state="expanded"
)

st.title("🌊 Proyek UAS: Dashboard Analisis & Prediksi Bencana Banjir")
st.markdown("### Analisis Data Bencana Alam berdasarkan Curah Hujan")

# --- 2. Muat Data dan Model ---

MODEL_FILE = 'model_prediksi_banjir.pkl'
DATA_FILE_PATH = os.path.join('/content/', 'dataset_banjir.csv')

# TIDAK ADA CACHE PADA MODEL
def load_model(file):
    try:
        return joblib.load(file)
    except Exception as e:
        st.sidebar.error(f"❌ Error memuat model ({file}): {e}")
        return None

model = load_model(MODEL_FILE)

# FUNGSI KRITIS: Muat dan Bersihkan Data (SAMA DENGAN PELATIHAN)
# TIDAK ADA CACHE PADA DATA
def load_and_clean_data(file_path):
    df = pd.DataFrame()

    try:
        df = pd.read_csv(file_path, sep=';')
    except Exception:
        return pd.DataFrame()

    df = df.dropna(axis=1, how='all')
    df = df.dropna(axis=0, how='all').reset_index(drop=True)

    if df.empty:
        return pd.DataFrame()

    df.columns = df.iloc[0]
    df = df[1:].reset_index(drop=True)

    try:
        df.columns = ['Kabupaten_Kota', 'Banjir', 'Banjir_Bandang', 'Curah_Hujan']
    except ValueError:
        return pd.DataFrame()

    kolom_numerik = ['Banjir', 'Banjir_Bandang', 'Curah_Hujan']
    for col in kolom_numerik:
        if df[col].dtype == 'object':
             df[col] = df[col].astype(str).str.replace(',', '.', regex=False)
        df[col] = pd.to_numeric(df[col], errors='coerce')

    df[kolom_numerik] = df[kolom_numerik].fillna(0)
    df['Total_Bencana'] = df['Banjir'] + df['Banjir_Bandang']

    return df

df = load_and_clean_data(DATA_FILE_PATH)

# --- 3. Bagian 1: Eksplorasi Data (EDA) ---
st.header("1. Hasil Analisis Data Eksplorasi (EDA)")

if df.empty:
     st.error("Gagal memuat data. Mohon periksa file 'dataset_banjir.csv' Anda.")
else:
    col1, col2 = st.columns(2)

    with col1:
        st.subheader("Kabupaten/Kota Paling Terdampak")
        df_top = df.sort_values(by='Total_Bencana', ascending=False).head(10)

        fig, ax = plt.subplots(figsize=(8, 6))
        sns.barplot(x='Total_Bencana', y='Kabupaten_Kota', data=df_top, palette='Reds_d', ax=ax)
        ax.set_xlabel('Jumlah Kejadian')
        ax.set_ylabel('Kabupaten/Kota')
        st.pyplot(fig)

    with col2:
        st.subheader("Hubungan Curah Hujan dan Total Bencana")
        fig, ax = plt.subplots(figsize=(8, 6))
        sns.scatterplot(x='Curah_Hujan', y='Total_Bencana', data=df, ax=ax, alpha=0.6)
        ax.set_xlabel('Curah Hujan (mm)')
        ax.set_ylabel('Total Bencana')
        st.pyplot(fig)

    st.subheader("Tabel Data Bersih")
    st.dataframe(df, use_container_width=True)


# --- 4. Bagian 2: Deployment Model Prediksi (RFR) ---
st.header("2. Deploy Model Prediksi (Random Forest)")

if model is not None and not df.empty:
    max_curah = df['Curah_Hujan'].max() + 50
    min_curah = df['Curah_Hujan'].min()
    median_curah = df['Curah_Hujan'].median()

    curah_hujan_input = st.slider(
        'Masukkan Nilai Curah Hujan (mm):',
        min_value=float(min_curah),
        max_value=float(max_curah),
        value=float(median_curah),
        step=0.1,
        key='slider_hujan'
    )

    if st.button("Lakukan Prediksi"):
        input_data = pd.DataFrame({'Curah_Hujan': [curah_hujan_input]})

        prediksi_mentah = model.predict(input_data)[0]
        prediksi_final = max(0, prediksi_mentah)

        st.success(f"✅ Prediksi Selesai!")
        st.metric(
            label=f"Total Prediksi Kejadian Bencana untuk Curah Hujan {curah_hujan_input:.2f} mm",
            value=f"{prediksi_final:.0f} Kejadian",
            delta=f"Menggunakan Model Random Forest Regressor"
        )
else:
    st.warning("Model belum dimuat atau data tidak ditemukan. Prediksi tidak dapat dilakukan.")

Overwriting app.py


In [18]:
%%writefile app.py

import streamlit as st
import pandas as pd
import joblib
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# --- 1. Konfigurasi Awal ---
st.set_page_config(
    page_title="Dashboard Prediksi Bencana Banjir",
    layout="wide",
    initial_sidebar_state="expanded"
)

st.title("🌊 Proyek UAS: Dashboard Analisis & Prediksi Bencana Banjir")
st.markdown("### Analisis Data Bencana Alam berdasarkan Curah Hujan")

# --- 2. Muat Data dan Model ---

MODEL_FILE = 'model_prediksi_banjir.pkl'
DATA_FILE_PATH = os.path.join('/content/', 'dataset_banjir.csv')

# TIDAK ADA CACHE
def load_model(file):
    try:
        return joblib.load(file)
    except Exception as e:
        st.sidebar.error(f"❌ Error memuat model ({file}): {e}")
        return None

model = load_model(MODEL_FILE)

# FUNGSI KRITIS: Muat dan Bersihkan Data (FINAL FIX DENGAN ILOC)
# TIDAK ADA CACHE
def load_and_clean_data(file_path):
    df = pd.DataFrame()

    try:
        df = pd.read_csv(file_path, sep=';')
    except Exception:
        return pd.DataFrame()

    df = df.dropna(axis=1, how='all')
    df = df.dropna(axis=0, how='all').reset_index(drop=True)

    if df.empty:
        return pd.DataFrame()

    df.columns = df.iloc[0]
    df = df[1:].reset_index(drop=True)

    # FIX KRITIS: Memilih HANYA 4 kolom pertama
    df = df.iloc[:, :4]

    try:
        df.columns = ['Kabupaten_Kota', 'Banjir', 'Banjir_Bandang', 'Curah_Hujan']
    except ValueError:
        return pd.DataFrame()

    kolom_numerik = ['Banjir', 'Banjir_Bandang', 'Curah_Hujan']
    for col in kolom_numerik:
        if df[col].dtype == 'object':
             df[col] = df[col].astype(str).str.replace(',', '.', regex=False)
        df[col] = pd.to_numeric(df[col], errors='coerce')

    df[kolom_numerik] = df[kolom_numerik].fillna(0)
    df['Total_Bencana'] = df['Banjir'] + df['Banjir_Bandang']

    return df

df = load_and_clean_data(DATA_FILE_PATH)

# ... (Kode EDA dan Prediksi Lanjutan)
st.header("1. Hasil Analisis Data Eksplorasi (EDA)")

if df.empty:
     st.error("Gagal memuat data. Mohon periksa file 'dataset_banjir.csv' Anda.")
else:
    col1, col2 = st.columns(2)

    with col1:
        st.subheader("Kabupaten/Kota Paling Terdampak")
        df_top = df.sort_values(by='Total_Bencana', ascending=False).head(10)

        fig, ax = plt.subplots(figsize=(8, 6))
        sns.barplot(x='Total_Bencana', y='Kabupaten_Kota', data=df_top, palette='Reds_d', ax=ax)
        ax.set_xlabel('Jumlah Kejadian')
        ax.set_ylabel('Kabupaten/Kota')
        st.pyplot(fig)

    with col2:
        st.subheader("Hubungan Curah Hujan dan Total Bencana")
        fig, ax = plt.subplots(figsize=(8, 6))
        sns.scatterplot(x='Curah_Hujan', y='Total_Bencana', data=df, ax=ax, alpha=0.6)
        ax.set_xlabel('Curah Hujan (mm)')
        ax.set_ylabel('Total Bencana')
        st.pyplot(fig)

    st.subheader("Tabel Data Bersih")
    st.dataframe(df, use_container_width=True)


st.header("2. Deploy Model Prediksi (Random Forest)")

if model is not None and not df.empty:
    max_curah = df['Curah_Hujan'].max() + 50
    min_curah = df['Curah_Hujan'].min()
    median_curah = df['Curah_Hujan'].median()

    curah_hujan_input = st.slider(
        'Masukkan Nilai Curah Hujan (mm):',
        min_value=float(min_curah),
        max_value=float(max_curah),
        value=float(median_curah),
        step=0.1,
        key='slider_hujan'
    )

    if st.button("Lakukan Prediksi"):
        input_data = pd.DataFrame({'Curah_Hujan': [curah_hujan_input]})

        prediksi_mentah = model.predict(input_data)[0]
        prediksi_final = max(0, prediksi_mentah)

        st.success(f"✅ Prediksi Selesai!")
        st.metric(
            label=f"Total Prediksi Kejadian Bencana untuk Curah Hujan {curah_hujan_input:.2f} mm",
            value=f"{prediksi_final:.0f} Kejadian",
            delta=f"Menggunakan Model Random Forest Regressor"
        )
else:
    st.warning("Model belum dimuat atau data tidak ditemukan. Prediksi tidak dapat dilakukan.")

Overwriting app.py


In [7]:
# Jalankan Streamlit di foreground pada port 8502
!streamlit run app.py --server.port 8502




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8502
  Network URL: http://172.28.0.12:8502
  External URL: http://35.194.217.85:8502

  Stopping...
  Stopping...


In [19]:
# Matikan semua proses lama di port 8502
!fuser -k 8502/tcp

In [21]:
from pyngrok import ngrok
import time

# GANTI DENGAN TOKEN NGROK ANDA!
NGROK_AUTH_TOKEN = "36s8Olf289m7OeUe0lOvMXpsMUA_tuN7aUEs2YxyNcTMLNUC"

# Bersihkan dan sambungkan kembali
ngrok.kill()
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

try:
    public_url = ngrok.connect(8502)

    print("\n---------------------------------------------------------")
    print("✅ DASHBOARD URL ANDA (Ngrok) TELAH DIPERBARUI:")
    print(public_url)
    print("---------------------------------------------------------")

except Exception as e:
    print(f"❌ GAGAL TOTAL MENGHUBUNGKAN NGROK. Error: {e}")
    print("Pastikan Server Streamlit (sel sebelumnya) sedang berjalan (muter) dan tidak ada error Python.")


---------------------------------------------------------
✅ DASHBOARD URL ANDA (Ngrok) TELAH DIPERBARUI:
NgrokTunnel: "https://graig-infundibular-inerrantly.ngrok-free.dev" -> "http://localhost:8502"
---------------------------------------------------------


In [ ]:
# JANGAN TAMBAHKAN NOHUP ATAU &
!streamlit run app.py --server.port 8502




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8502
  Network URL: http://172.28.0.12:8502
  External URL: http://35.194.217.85:8502

/content/app.py:88: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x='Total_Bencana', y='Kabupaten_Kota', data=df_top, palette='Reds_d', ax=ax)
2025-12-15 19:10:10.047 Please replace `use_container_width` with `width`.

`use_container_width` will be removed after 2025-12-31.

For `use_container_width=True`, use `width='stretch'`. For `use_container_width=False`, use `width='content'`.
/content/app.py:88: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x='Total_Bencana', y='Kabupaten_Kota', data=df_top, palette='Reds_d',